In [48]:
# import libraries
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go # for candlestick chart
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import sys
import os
from sqlalchemy import create_engine, text
from utils.config_utils import load_config

sys.path.append(os.path.abspath('../src'))  # Add src directory to path

# Get database configuration using the correct path to config.yaml
config = load_config("../config.yaml")  # Go up one directory to find config.yaml
db_config = config['database']

In [49]:
# Create SQLAlchemy engine
engine = create_engine(f"postgresql://{db_config['user']}:{db_config['password']}@"
                      f"{db_config['host']}:{db_config['port']}/{db_config['dbname']}")

# Read into pandas DataFrames using SQLAlchemy 
with engine.connect() as connection:
    stocks_df = pd.read_sql_query(text("SELECT * FROM stocks_data"), connection)
    indices_df = pd.read_sql_query(text("SELECT * FROM indices_data"), connection)


print("\nStocks DataFrame Info:")
print(stocks_df.info())

print("\nIndices DataFrame Info:")
print(indices_df.info())


Stocks DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25501 entries, 0 to 25500
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   datetime   25494 non-null  datetime64[ns]
 1   open       25494 non-null  float64       
 2   high       25494 non-null  float64       
 3   low        25494 non-null  float64       
 4   close      25494 non-null  float64       
 5   adj_close  25494 non-null  float64       
 6   volume     25494 non-null  float64       
 7   symbol     25501 non-null  object        
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 1.6+ MB
None

Indices DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13095 entries, 0 to 13094
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   datetime   13091 non-null  datetime64[ns]
 1   open       13091 non-null  float64       
 

In [50]:
display(stocks_df)

,datetime,open,high,low,close,adj_close,volume,symbol
0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,AAPL
1,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.447412,493729600.0,AAPL
2,2010-01-05,7.664286,7.699643,7.616071,7.656429,6.458559,601904770.0,AAPL
3,2010-01-06,7.656429,7.686786,7.526786,7.534643,6.355827,552160000.0,AAPL
4,2010-01-07,7.562500,7.571429,7.466071,7.520714,6.344078,477131200.0,AAPL
...,...,...,...,...,...,...,...,...
25496,2024-11-12,342.740000,345.840000,323.310000,328.490000,328.490000,155726000.0,TSLA
25497,2024-11-13,335.850000,344.600000,322.500000,330.240000,330.240000,125405600.0,TSLA
25498,2024-11-14,327.690000,329.980000,310.370000,311.180000,311.180000,120726096.0,TSLA
25499,2024-11-15,310.570000,324.680000,309.220000,320.720000,320.720000,114440304.0,TSLA


In [51]:
stocks_df.dtypes

datetime     datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
adj_close           float64
volume              float64
symbol               object
dtype: object

In [52]:
stocks_df.isna().sum() # the reason might be the process of adding data to the database

datetime     7
open         7
high         7
low          7
close        7
adj_close    7
volume       7
symbol       0
dtype: int64

In [53]:
# 1. Get total count of NaN values in each column
print("NaN count in each column:")
print(stocks_df.isna().sum())

# 2. Get percentage of NaN values in each column
print("\nPercentage of NaN values in each column:")
print((stocks_df.isna().sum() / len(stocks_df)) * 100)

# 3. Show rows with any NaN values
print("\nSample of rows containing NaN values:")
print(stocks_df[stocks_df.isna().any(axis=1)].head())

# 4. Count of rows with NaN values
print("\nTotal rows with any NaN value:")
print(stocks_df.isna().any(axis=1).sum())

# 5. Check NaN values by symbol
print("\nNaN values by symbol:")
for symbol in stocks_df['symbol'].unique():
    symbol_df = stocks_df[stocks_df['symbol'] == symbol]
    print(f"\n{symbol}:")
    print(symbol_df.isna().sum())

# 6. Detailed info about the DataFrame
print("\nDataFrame Info:")
print(stocks_df.info())

NaN count in each column:
datetime     7
open         7
high         7
low          7
close        7
adj_close    7
volume       7
symbol       0
dtype: int64

Percentage of NaN values in each column:
datetime     0.02745
open         0.02745
high         0.02745
low          0.02745
close        0.02745
adj_close    0.02745
volume       0.02745
symbol       0.00000
dtype: float64

Sample of rows containing NaN values:
    datetime  open  high  low  close  adj_close  volume symbol
0        NaT   NaN   NaN  NaN    NaN        NaN     NaN   AAPL
95       NaT   NaN   NaN  NaN    NaN        NaN     NaN   AMZN
190      NaT   NaN   NaN  NaN    NaN        NaN     NaN   AVGO
285      NaT   NaN   NaN  NaN    NaN        NaN     NaN   META
379      NaT   NaN   NaN  NaN    NaN        NaN     NaN   MSFT

Total rows with any NaN value:
7

NaN values by symbol:

AAPL:
datetime     1
open         1
high         1
low          1
close        1
adj_close    1
volume       1
symbol       0
dtype: int64

A

* kayıp verilerin dataları birbiri ardına eklerken oluştuğunu düşünüyorum
* AAPL, MSFT, NVDA has 6 missing values while AMZN, AVGO and META has 1 missing value

## Basic summary statistics

In [54]:
# Calculate and display summary statistics for each symbol
for symbol in stocks_df['symbol'].unique():
    symbol_data = stocks_df[stocks_df['symbol'] == symbol]
    print(f"\nSummary Statistics for {symbol}:")
    print(symbol_data.describe())
    print("-" * 80)  


Summary Statistics for AAPL:
              open         high          low        close    adj_close  \
count  3745.000000  3745.000000  3745.000000  3745.000000  3745.000000   
mean     68.358177    69.078041    67.672518    68.407055    66.225740   
std      63.641842    64.316861    63.017848    63.702540    64.043381   
min       6.870357     7.000000     6.794643     6.858929     5.785831   
25%      20.414286    20.540714    20.228930    20.410713    17.405085   
50%      36.885000    37.207500    36.505000    36.982500    34.727190   
75%     127.210000   128.460000   125.870000   126.900000   124.682526   
max     236.480000   237.490000   234.450000   236.480000   236.220110   

             volume  
count  3.745000e+03  
mean   2.313459e+08  
std    2.183010e+08  
min    2.404830e+07  
25%    8.614160e+07  
50%    1.417080e+08  
75%    3.100692e+08  
max    1.880998e+09  
--------------------------------------------------------------------------------

Summary Statistics for 

In [55]:
# Calculate and display summary statistics for each symbol
for symbol in stocks_df['symbol'].unique():
    symbol_data = stocks_df[stocks_df['symbol'] == symbol]
    print(f"\nSummary Statistics for {symbol}:")
    print(symbol_data.info())
    print("-" * 80) 


Summary Statistics for AAPL:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3746 entries, 0 to 3751
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   datetime   3745 non-null   datetime64[ns]
 1   open       3745 non-null   float64       
 2   high       3745 non-null   float64       
 3   low        3745 non-null   float64       
 4   close      3745 non-null   float64       
 5   adj_close  3745 non-null   float64       
 6   volume     3745 non-null   float64       
 7   symbol     3746 non-null   object        
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 263.4+ KB
None
--------------------------------------------------------------------------------

Summary Statistics for AMZN:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3746 entries, 95 to 7496
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0  

In [56]:
# Calculate and display summary statistics for each symbol
for symbol in stocks_df['symbol'].unique():
    symbol_data = stocks_df[stocks_df['symbol'] == symbol]
    print(f"\nMissing values for {symbol}:")
    print(symbol_data.isna().sum())
    print("-" * 80) 


Missing values for AAPL:
datetime     1
open         1
high         1
low          1
close        1
adj_close    1
volume       1
symbol       0
dtype: int64
--------------------------------------------------------------------------------

Missing values for AMZN:
datetime     1
open         1
high         1
low          1
close        1
adj_close    1
volume       1
symbol       0
dtype: int64
--------------------------------------------------------------------------------

Missing values for AVGO:
datetime     1
open         1
high         1
low          1
close        1
adj_close    1
volume       1
symbol       0
dtype: int64
--------------------------------------------------------------------------------

Missing values for META:
datetime     1
open         1
high         1
low          1
close        1
adj_close    1
volume       1
symbol       0
dtype: int64
--------------------------------------------------------------------------------

Missing values for MSFT:
datetime     1

In [57]:
for symbol in stocks_df['symbol'].unique():
    symbol_data = stocks_df[stocks_df['symbol'] == symbol]
    print(f"\nDuplicated values for {symbol}:")
    print(symbol_data.duplicated().sum())
    print("-" * 80) 


Duplicated values for AAPL:
0
--------------------------------------------------------------------------------

Duplicated values for AMZN:
0
--------------------------------------------------------------------------------

Duplicated values for AVGO:
0
--------------------------------------------------------------------------------

Duplicated values for META:
0
--------------------------------------------------------------------------------

Duplicated values for MSFT:
0
--------------------------------------------------------------------------------

Duplicated values for NVDA:
0
--------------------------------------------------------------------------------

Duplicated values for TSLA:
0
--------------------------------------------------------------------------------


In [61]:
stocks_df = stocks_df.dropna()
stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25494 entries, 1 to 25500
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   datetime   25494 non-null  datetime64[ns]
 1   open       25494 non-null  float64       
 2   high       25494 non-null  float64       
 3   low        25494 non-null  float64       
 4   close      25494 non-null  float64       
 5   adj_close  25494 non-null  float64       
 6   volume     25494 non-null  float64       
 7   symbol     25494 non-null  object        
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 1.8+ MB


In [62]:
stocks_df

,datetime,open,high,low,close,adj_close,volume,symbol
1,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.447412,493729600.0,AAPL
2,2010-01-05,7.664286,7.699643,7.616071,7.656429,6.458559,601904770.0,AAPL
3,2010-01-06,7.656429,7.686786,7.526786,7.534643,6.355827,552160000.0,AAPL
4,2010-01-07,7.562500,7.571429,7.466071,7.520714,6.344078,477131200.0,AAPL
5,2010-01-08,7.510714,7.571429,7.466429,7.570714,6.386254,447610820.0,AAPL
...,...,...,...,...,...,...,...,...
25496,2024-11-12,342.740000,345.840000,323.310000,328.490000,328.490000,155726000.0,TSLA
25497,2024-11-13,335.850000,344.600000,322.500000,330.240000,330.240000,125405600.0,TSLA
25498,2024-11-14,327.690000,329.980000,310.370000,311.180000,311.180000,120726096.0,TSLA
25499,2024-11-15,310.570000,324.680000,309.220000,320.720000,320.720000,114440304.0,TSLA
